pre-train with AUTSL dataset. The input numpy files are located in /home/ibmelab/Documents/GG/VSLRecognition/AUTSL/AAGCN

The AUTSL dataset has 226 classes in total. So the code below defines num_label as 226, not the VSL num_label (which is defined in the bottom)

In [1]:
'''
from torch.utils.data import DataLoader
import torch
from torchinfo import summary
from feeder import FeederINCLUDE
from aagcn import Model
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger  # Importing here
from pytorch_lightning.callbacks import ModelCheckpoint
import wandb
from augumentation import Rotate, Compose
from torch.utils.data import random_split


if __name__ == '__main__':

    # Hyper parameter tuning : batch_size, learning_rate, weight_decay
    config = {'batch_size': 150, 'learning_rate': 0.0137296, 'weight_decay': 0.000150403}
    
    # Load device
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Initialize wandb
    wandb.finish()
    wandb.init(project="GCN_VSL", config=config)  
    wandb_config = wandb.config  # Access the config parameters

    try:
    # Your training or evaluation code here
        print("WandB initialized successfully.")

    finally:
    # Fnish the WandB run
        wandb.finish()

    # Load model
    model = Model(num_class=226, num_point=46, num_person=1, in_channels=2,
                  graph_args={"layout": "mediapipe_two_hand", "strategy": "spatial"},
                  learning_rate=wandb_config.learning_rate, weight_decay=wandb_config.weight_decay)

    # Callback PL
    callbacks = [
        ModelCheckpoint(
            dirpath="checkpoints",
            monitor="valid_loss",
            mode="min",
            every_n_epochs=2,
            filename='{epoch}-{valid_loss:.2f}-{valid_accuracy:.2f}-autsl-aagcn'
        ),
    ]

    # Augmentation
    transforms = Compose([
        Rotate(15, 80, 25, (0.5, 0.5))
    ])

    %cd /home/ibmelab/Documents/GG/VSLRecognition/AUTSL/AAGCN
    # Dataset class
    train_dataset = FeederINCLUDE(data_path=f"autsl_train_data_preprocess.npy", label_path=f"train_label_preprocess.npy",
                                   transform=transforms)
    test_dataset = FeederINCLUDE(data_path=f"autsl_test_data_preprocess.npy", label_path=f"test_label_preprocess.npy")
    valid_dataset = FeederINCLUDE(data_path=f"autsl_valid_data_preprocess.npy", label_path=f"valid_label_preprocess.npy")

    # DataLoader
    train_dataloader = DataLoader(train_dataset, batch_size=wandb_config.batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=wandb_config.batch_size, shuffle=False)
    val_dataloader = DataLoader(valid_dataset, batch_size=wandb_config.batch_size, shuffle=False)

    # Wandb Logger
    wandb_logger = WandbLogger(log_model='all')

    %cd /media/ibmelab/ibme21/Test
    # Trainer PL
    trainer = pl.Trainer(max_epochs=120, accelerator="auto", check_val_every_n_epoch=1,
                         devices=1, callbacks=callbacks, logger=wandb_logger)  # Added logger here

    trainer.fit(model, train_dataloader, val_dataloader)

    # Optional: Uncomment this when you want to test
    # trainer.test(model, test_dataloader, ckpt_path="checkpoints/your_checkpoint.ckpt", verbose=True)
'''
    


'\nfrom torch.utils.data import DataLoader\nimport torch\nfrom torchinfo import summary\nfrom feeder import FeederINCLUDE\nfrom aagcn import Model\nimport pytorch_lightning as pl\nfrom pytorch_lightning.loggers import WandbLogger  # Importing here\nfrom pytorch_lightning.callbacks import ModelCheckpoint\nimport wandb\nfrom augumentation import Rotate, Compose\nfrom torch.utils.data import random_split\n\n\nif __name__ == \'__main__\':\n\n    # Hyper parameter tuning : batch_size, learning_rate, weight_decay\n    config = {\'batch_size\': 150, \'learning_rate\': 0.0137296, \'weight_decay\': 0.000150403}\n    \n    # Load device\n    device = "cuda" if torch.cuda.is_available() else "cpu"\n\n    # Initialize wandb\n    wandb.finish()\n    wandb.init(project="GCN_VSL", config=config)  \n    wandb_config = wandb.config  # Access the config parameters\n\n    try:\n    # Your training or evaluation code here\n        print("WandB initialized successfully.")\n\n    finally:\n    # Fnish the W

In [2]:
import pandas as pd
import mediapipe as mp
import cv2
from collections import defaultdict
from joblib import Parallel, delayed
from tqdm import tqdm
import ast
import os
import csv
import re
from sklearn.model_selection import KFold
import numpy as np

Load the videos to videos_list.csv (columns: file (path), label, gloss, video name, actor)

In [3]:
folder_path = r'10 class 28 actor (center new)'
csv_file_path = 'videos_list.csv'
labels_file_path = '1_1000_label.csv'
final_file_path = 'temp_videos_list.csv'

label_to_gloss = {}
with open(labels_file_path, mode='r', encoding='utf-8') as labels_file:
    csv_reader = csv.DictReader(labels_file)
    for row in csv_reader:
        label = int(row['id_label_in_documents'])
        gloss = row['name']
        label_to_gloss[label] = gloss

with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['file', 'label', 'gloss', 'video_name', 'actor'])

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.mp4', '.mkv', '.avi', '.mov', '.flv', '.wmv')):
            actor = filename.split('_')[0]
            
            match = re.search(r'_(\d+)\.', filename)
            if match:
                label = int(match.group(1))
                gloss = label_to_gloss.get(label, 'Unknown')
            else:
                label = 'N/A'
                gloss = 'Unknown'

            if label != 200:
                full_filename = os.path.join(folder_path, filename)
                csv_writer.writerow([full_filename, label, gloss, filename, actor]) 

print(f'Video names have been written to {csv_file_path}')

# Find min label
with open(csv_file_path, mode='r', newline='', encoding='utf-8') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    labels = [int(row["label"]) for row in csv_reader if row["label"].isdigit()]  
    min_label = min(labels) if labels else None

print("Minimum label:", min_label)

# Normalize labels
with open(csv_file_path, mode='r', newline='', encoding='utf-8') as csv_file, \
     open(final_file_path, mode='w', newline='', encoding='utf-8') as final_file:
    
    csv_reader = csv.DictReader(csv_file)
    fieldnames = csv_reader.fieldnames
    
    csv_writer = csv.DictWriter(final_file, fieldnames=fieldnames)
    csv_writer.writeheader()
    
    for row in csv_reader:
        if row['label'].isdigit():  # Check if label is a digit before converting
            row['label'] = str(int(row['label']) - min_label)  # Normalize and convert back to string
        csv_writer.writerow(row)

# Replace the original file with the updated file
os.replace(final_file_path, csv_file_path)

print("Labels have been updated and saved.")


Video names have been written to videos_list.csv
Minimum label: 20
Labels have been updated and saved.


Number of labels in the dataset

In [4]:
num_labels = len(set(labels))
print(num_labels)
print(set(labels))

10
{20, 21, 22, 23, 24, 25, 26, 27, 28, 29}


Extract keypoints

In [5]:
# Extract keypoints version Parallel
import pandas as pd
import mediapipe as mp
import cv2
import os
from collections import defaultdict
from joblib import Parallel, delayed
from tqdm import tqdm

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

hand_landmarks = ['INDEX_FINGER_DIP', 'INDEX_FINGER_MCP', 'INDEX_FINGER_PIP', 'INDEX_FINGER_TIP', 
                  'MIDDLE_FINGER_DIP', 'MIDDLE_FINGER_MCP', 'MIDDLE_FINGER_PIP', 'MIDDLE_FINGER_TIP', 
                  'PINKY_DIP', 'PINKY_MCP', 'PINKY_PIP', 'PINKY_TIP', 'RING_FINGER_DIP', 'RING_FINGER_MCP', 
                  'RING_FINGER_PIP', 'RING_FINGER_TIP', 'THUMB_CMC', 'THUMB_IP', 'THUMB_MCP', 'THUMB_TIP', 'WRIST']
pose_landmarks = ['LEFT_ANKLE', 'LEFT_EAR', 'LEFT_ELBOW', 'LEFT_EYE', 'LEFT_EYE_INNER', 'LEFT_EYE_OUTER', 
                  'LEFT_FOOT_INDEX', 'LEFT_HEEL', 'LEFT_HIP', 'LEFT_INDEX', 'LEFT_KNEE', 'LEFT_PINKY', 
                  'LEFT_SHOULDER', 'LEFT_THUMB', 'LEFT_WRIST', 'MOUTH_LEFT', 'MOUTH_RIGHT', 'NOSE', 
                  'RIGHT_ANKLE', 'RIGHT_EAR', 'RIGHT_ELBOW', 'RIGHT_EYE', 'RIGHT_EYE_INNER', 'RIGHT_EYE_OUTER', 
                  'RIGHT_FOOT_INDEX', 'RIGHT_HEEL', 'RIGHT_HIP', 'RIGHT_INDEX', 'RIGHT_KNEE', 'RIGHT_PINKY', 
                  'RIGHT_SHOULDER', 'RIGHT_THUMB', 'RIGHT_WRIST']

def extract_keypoint(video_path, label, actor):
    cap = cv2.VideoCapture(video_path)
    
    keypoint_dict = defaultdict(list)
    count = 0

    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            count += 1
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = holistic.process(image)

            if results.right_hand_landmarks:
                for idx, landmark in enumerate(results.right_hand_landmarks.landmark): 
                    keypoint_dict[f"{hand_landmarks[idx]}_right_x"].append(landmark.x)
                    keypoint_dict[f"{hand_landmarks[idx]}_right_y"].append(landmark.y)
                    keypoint_dict[f"{hand_landmarks[idx]}_right_z"].append(landmark.z)
            else:
                for idx in range(len(hand_landmarks)):
                    keypoint_dict[f"{hand_landmarks[idx]}_right_x"].append(0)
                    keypoint_dict[f"{hand_landmarks[idx]}_right_y"].append(0)
                    keypoint_dict[f"{hand_landmarks[idx]}_right_z"].append(0)

            if results.left_hand_landmarks:
                for idx, landmark in enumerate(results.left_hand_landmarks.landmark): 
                    keypoint_dict[f"{hand_landmarks[idx]}_left_x"].append(landmark.x)
                    keypoint_dict[f"{hand_landmarks[idx]}_left_y"].append(landmark.y)
                    keypoint_dict[f"{hand_landmarks[idx]}_left_z"].append(landmark.z)
            else:
                for idx in range(len(hand_landmarks)):
                    keypoint_dict[f"{hand_landmarks[idx]}_left_x"].append(0)
                    keypoint_dict[f"{hand_landmarks[idx]}_left_y"].append(0)
                    keypoint_dict[f"{hand_landmarks[idx]}_left_z"].append(0)

            if results.pose_landmarks:
                for idx, landmark in enumerate(results.pose_landmarks.landmark): 
                    keypoint_dict[f"{pose_landmarks[idx]}_x"].append(landmark.x)
                    keypoint_dict[f"{pose_landmarks[idx]}_y"].append(landmark.y)
                    keypoint_dict[f"{pose_landmarks[idx]}_z"].append(landmark.z)
            else:
                for idx in range(len(pose_landmarks)):
                    keypoint_dict[f"{pose_landmarks[idx]}_x"].append(0)
                    keypoint_dict[f"{pose_landmarks[idx]}_y"].append(0)
                    keypoint_dict[f"{pose_landmarks[idx]}_z"].append(0)

        keypoint_dict["frame"] = count
        keypoint_dict["video_path"] = video_path
        keypoint_dict["label"] = label
        keypoint_dict["actor"] = actor

        return keypoint_dict

def process_videos():
    csv_file = f"videos_list.csv"
    data = pd.read_csv(csv_file)

    keypoints_list = Parallel(n_jobs=-1)(  # Chạy song song với số lượng core tối đa
        delayed(extract_keypoint)(row['file'], row['label'], row['actor']) for index, row in tqdm(data.iterrows(), total=len(data), desc="Processing videos", leave=False)
    )

    keypoints_df = pd.DataFrame(keypoints_list)
    keypoints_df.to_csv(f"vsl{num_labels}_keypoints.csv", index=False)

if __name__ == '__main__':
    process_videos()


Interpolation

In [6]:
import pandas as pd
import numpy as np
import ast
from tqdm import tqdm

def find_index(array):
    for i, num in enumerate(array):
        if num != 0:
            return i

def curl_skeleton(array):
    if sum(array) == 0:
        return array
    for i, location in enumerate(array):
        if location != 0:
            continue
        else:
            if i == 0 or i == len(array) - 1:
                continue
            else:
                if array[i + 1] != 0:
                    array[i] = float((array[i - 1] + array[i + 1]) / 2)
                else:
                    if sum(array[i:]) == 0:
                        continue
                    else:
                        j = find_index(array[i + 1:])
                        array[i] = float(((1 + j) * array[i - 1] + 1 * array[i + 1 + j]) / (2 + j))
    return array

def interpolate_keypoints(input_file, output_file, body_identifiers):
    train_data = pd.read_csv(input_file)
    output_df = train_data.copy()

    for index, video in tqdm(train_data.iterrows(), total=train_data.shape[0]):
        for identifier in body_identifiers:
            # Interpolate the x and y keypoints
            x_values = curl_skeleton(ast.literal_eval(video[identifier + "_x"]))
            y_values = curl_skeleton(ast.literal_eval(video[identifier + "_y"]))

            output_df.at[index, identifier + "_x"] = str(x_values)
            output_df.at[index, identifier + "_y"] = str(y_values)

    output_df.to_csv(output_file, index=False)
    print(f"Interpolated keypoints saved to {output_file}")

if __name__ == "__main__":
    input_file_path = f"vsl{num_labels}_keypoints.csv"
    output_file_path = f"vsl{num_labels}_interpolated_keypoints.csv"

    hand_landmarks = [
        'INDEX_FINGER_DIP', 'INDEX_FINGER_MCP', 'INDEX_FINGER_PIP', 'INDEX_FINGER_TIP', 
        'MIDDLE_FINGER_DIP', 'MIDDLE_FINGER_MCP', 'MIDDLE_FINGER_PIP', 'MIDDLE_FINGER_TIP', 
        'PINKY_DIP', 'PINKY_MCP', 'PINKY_PIP', 'PINKY_TIP', 
        'RING_FINGER_DIP', 'RING_FINGER_MCP', 'RING_FINGER_PIP', 'RING_FINGER_TIP', 
        'THUMB_CMC', 'THUMB_IP', 'THUMB_MCP', 'THUMB_TIP', 'WRIST'
    ]
    HAND_IDENTIFIERS = [id + "_right" for id in hand_landmarks] + [id + "_left" for id in hand_landmarks]
    POSE_IDENTIFIERS = ["RIGHT_SHOULDER", "LEFT_SHOULDER", "LEFT_ELBOW", "RIGHT_ELBOW"]
    body_identifiers = HAND_IDENTIFIERS + POSE_IDENTIFIERS 

    interpolate_keypoints(input_file_path, output_file_path, body_identifiers)

    # Load interpolated data for further processing
    train_data = pd.read_csv(output_file_path)
    frames = 80

    data = []
    labels = []

    for video_index, video in tqdm(train_data.iterrows(), total=train_data.shape[0]):
        T = len(ast.literal_eval(video["INDEX_FINGER_DIP_right_x"]))
        current_row = np.empty(shape=(2, T, len(body_identifiers), 1))

        for index, identifier in enumerate(body_identifiers):
            data_keypoint_preprocess_x = ast.literal_eval(video[identifier + "_x"])
            current_row[0, :, index, :] = np.asarray(data_keypoint_preprocess_x).reshape(T, 1)

            data_keypoint_preprocess_y = ast.literal_eval(video[identifier + "_y"])
            current_row[1, :, index, :] = np.asarray(data_keypoint_preprocess_y).reshape(T, 1)

        if T < frames:
            target = np.zeros(shape=(2, frames, len(body_identifiers), 1))
            target[:, :T, :, :] = current_row
        else:
            target = current_row[:, :frames, :, :]

        data.append(target)
        labels.append(int(video["label"]))

    # Save preprocessed data and labels
    keypoint_data = np.stack(data, axis=0)
    label_data = np.stack(labels, axis=0)
    np.save(f'vsl{num_labels}_data_preprocess.npy', keypoint_data)
    np.save(f'vsl{num_labels}_label_preprocess.npy', label_data)
    print("Data processing and saving completed.")


100%|██████████| 280/280 [00:04<00:00, 67.80it/s]


Interpolated keypoints saved to vsl10_interpolated_keypoints.csv


100%|██████████| 280/280 [00:03<00:00, 91.60it/s] 

Data processing and saving completed.


In [7]:
import numpy as np
a = np.load(f'vsl{num_labels}_data_preprocess.npy')
b = np.load(f'vsl{num_labels}_label_preprocess.npy')

print(a.shape)
print(b.shape)

(280, 2, 80, 46, 1)
(280,)


Do K-Folds and store the keypoints in numpy files

In [8]:
from sklearn.model_selection import KFold
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

def k_fold_cross_validation(train_data, keypoint_data, label_data, num_labels, k_folds, destination_folder="numpy_files"):
    os.makedirs(destination_folder, exist_ok=True)

    actors = train_data['actor'].unique()
    print(f"Number of actors: {len(actors)}")
    print('-----------------------------------------------------')

    kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

    actor_to_indices = {actor: train_data.index[train_data['actor'] == actor].tolist() for actor in actors}
    folds = [[] for _ in range(k_folds)]

    for fold, (train_actors, test_actors) in enumerate(kf.split(actors)):
        train_actors = actors[train_actors]
        test_actors = actors[test_actors]
        
        for actor in test_actors:
            folds[fold].extend(actor_to_indices[actor])

        tqdm.write(f"Fold {fold+1}: {len(folds[fold])} test samples")

    # Iterate over each fold to create train-test splits
    for fold in range(k_folds):
        test_indices = folds[fold]
        train_indices = [idx for f in range(k_folds) if f != fold for idx in folds[f]]

        X_train, X_test = keypoint_data[train_indices], keypoint_data[test_indices]
        y_train = np.array(label_data[train_indices], dtype=np.int64)
        y_test = np.array(label_data[test_indices], dtype=np.int64)

        # Save train and test data for this fold
        np.save(os.path.join(destination_folder, f'vsl{num_labels}_data_fold{fold+1}_train.npy'), X_train)
        np.save(os.path.join(destination_folder, f'vsl{num_labels}_label_fold{fold+1}_train.npy'), y_train)
        np.save(os.path.join(destination_folder, f'vsl{num_labels}_data_fold{fold+1}_test.npy'), X_test)
        np.save(os.path.join(destination_folder, f'vsl{num_labels}_label_fold{fold+1}_test.npy'), y_test)

        tqdm.write(f"Processed and saved vsl{num_labels} fold {fold+1} successfully.")

if __name__ == "__main__":
    input_file_path = f"vsl{num_labels}_interpolated_keypoints.csv"
    train_data = pd.read_csv(input_file_path)

    keypoint_data = np.load(f'vsl{num_labels}_data_preprocess.npy')
    label_data = np.load(f'vsl{num_labels}_label_preprocess.npy')

    num_labels = len(np.unique(label_data))

    k_folds = 10
    k_fold_cross_validation(train_data, keypoint_data, label_data, num_labels, k_folds)


Number of actors: 28
-----------------------------------------------------
Fold 1: 30 test samples
Fold 2: 29 test samples
Fold 3: 30 test samples
Fold 4: 30 test samples
Fold 5: 30 test samples
Fold 6: 30 test samples
Fold 7: 31 test samples
Fold 8: 30 test samples
Fold 9: 20 test samples
Fold 10: 20 test samples
Processed and saved vsl10 fold 1 successfully.
Processed and saved vsl10 fold 2 successfully.
Processed and saved vsl10 fold 3 successfully.
Processed and saved vsl10 fold 4 successfully.
Processed and saved vsl10 fold 5 successfully.
Processed and saved vsl10 fold 6 successfully.
Processed and saved vsl10 fold 7 successfully.
Processed and saved vsl10 fold 8 successfully.
Processed and saved vsl10 fold 9 successfully.
Processed and saved vsl10 fold 10 successfully.


In [9]:
import numpy as np
a = np.load(f'numpy_files/vsl{num_labels}_data_fold2_test.npy')
b = np.load(f'numpy_files/vsl{num_labels}_data_fold2_train.npy')

print(a.shape)
print(b.shape)

(29, 2, 80, 46, 1)
(251, 2, 80, 46, 1)


train directly with different folds

In [ ]:

import os
import numpy as np
import torch
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from feeder import FeederINCLUDE
from aagcn import Model
from augumentation import Rotate, Compose

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

if __name__ == '__main__':
    k_folds = 2
    config = {'batch_size': 128, 'learning_rate': 0.0137296, 'weight_decay': 0.000150403}
    
    device = "cuda" if torch.cuda.is_available() else "cpu"

    best_accuracy = 0.0
    best_fold = -1

    for fold in range(k_folds):
        print(f"Starting fold {fold + 1}/{k_folds}")
        train_data_path = os.path.join("numpy_files", f'vsl{num_labels}_data_fold{fold+1}_train.npy')
        train_label_path = os.path.join("numpy_files", f'vsl{num_labels}_label_fold{fold+1}_train.npy')
        val_data_path = os.path.join("numpy_files", f'vsl{num_labels}_data_fold{fold+1}_test.npy')
        val_label_path = os.path.join("numpy_files", f'vsl{num_labels}_label_fold{fold+1}_test.npy')

        transforms = Compose([
            Rotate(15, 80, 25, (0.5, 0.5))
        ])

        train_dataset = FeederINCLUDE(
            data_path=train_data_path,
            label_path=train_label_path,
            transform=transforms
        )
        val_dataset = FeederINCLUDE(
            data_path=val_data_path,
            label_path=val_label_path
        )

        train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
        val_dataloader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)

        model = Model(num_class=num_labels, num_point=46, num_person=1, in_channels=2,
                      graph_args={"layout": "mediapipe_two_hand", "strategy": "spatial"},
                      learning_rate=config['learning_rate'], weight_decay=config['weight_decay'])

        callbacks = [
            ModelCheckpoint(
                dirpath="checkpoints",
                monitor="valid_accuracy",
                mode="max",
                every_n_epochs=2,
                #filename=f'{{epoch}}-{{valid_accuracy:.2f}}-vsl{num_labels}-aagcn-fold={fold}'
                filename=f'vsl{num_labels}-aagcn-fold={fold+1}'
            ),
        ]

        trainer = pl.Trainer(max_epochs=2, accelerator="auto", check_val_every_n_epoch=1,
                             devices=1, callbacks=callbacks)

        trainer.fit(model, train_dataloader, val_dataloader)

        # Get the best validation accuracy for this fold
        val_accuracy = trainer.callback_metrics['valid_accuracy'].item()  # Access the validation accuracy
        print(f"Fold {fold + 1} finished with validation accuracy: {val_accuracy:.4f}")

        # Update the best accuracy and fold index if this fold's accuracy is higher
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_fold = fold + 1  # Store fold number (1-indexed)

    # Print the highest accuracy and the corresponding fold
    print(f"The highest validation accuracy achieved is {best_accuracy:.4f} from fold {best_fold}.")


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\Hngk\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default

   | Name    | Type               | Params | Mode 
--------------------------------------------------------
0  | data_bn | BatchNorm1d        | 184    | train
1  | l1      | TCN_GCN_unit       | 52.4 K | train
2  | l2      | TCN_GCN_unit       | 69.9 K | train
3  | l3      | TCN_GCN_unit       | 69.9 K | train
4  | l4      | TCN_G

Starting fold 1/2
[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (10, 10), (11, 11), (12, 12), (13, 13), (14, 14), (15, 15), (16, 16), (17, 17), (18, 18), (19, 19), (20, 20), (21, 21), (22, 22), (23, 23), (24, 24), (25, 25), (26, 26), (27, 27), (28, 28), (29, 29), (30, 30), (31, 31), (32, 32), (33, 33), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 39), (40, 40), (41, 41), (42, 42), (43, 43), (44, 44), (45, 45), (41, 37), (37, 39), (39, 38), (38, 40), (41, 22), (22, 23), (23, 21), (21, 24), (22, 26), (26, 27), (27, 25), (25, 28), (26, 34), (34, 35), (35, 33), (33, 36), (41, 30), (30, 31), (31, 29), (29, 32), (20, 16), (16, 18), (18, 17), (17, 19), (20, 1), (1, 2), (2, 0), (0, 3), (1, 5), (5, 6), (6, 4), (4, 7), (5, 13), (13, 14), (14, 12), (12, 15), (20, 9), (9, 10), (10, 8), (8, 11), (42, 45), (45, 20), (42, 43), (43, 44), (44, 41)]
Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]

c:\Users\Hngk\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


c:\Users\Hngk\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\Users\Hngk\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\loops\fit_loop.py:310: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 0: 100%|██████████| 2/2 [02:55<00:00,  0.01it/s, v_num=0, train_accuracy_step=0.0749, train_loss_step=32.70]

In [ ]:

print(f"The highest validation accuracy achieved of vsl{num_labels} is {best_accuracy:.4f} from fold {best_fold}.")

train based on AUTSL with different folds

In [ ]:
'''
import os
import numpy as np
import torch
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from feeder import FeederINCLUDE
from aagcn import Model
from augumentation import Rotate, Compose
from pytorch_lightning.utilities.migration import pl_legacy_patch

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

if __name__ == '__main__':
    k_folds = 10 
    config = {'batch_size': 128, 'learning_rate': 0.0137296, 'weight_decay': 0.000150403}
    
    device = "cuda" if torch.cuda.is_available() else "cpu"

    best_accuracy = 0.0
    best_fold = -1

    for fold in range(k_folds):
        print(f"Starting fold {fold + 1}/{k_folds}")
        train_data_path = os.path.join("numpy_files", f'vsl{num_labels}_data_fold{fold+1}_train.npy')
        train_label_path = os.path.join("numpy_files", f'vsl{num_labels}_label_fold{fold+1}_train.npy')
        val_data_path = os.path.join("numpy_files", f'vsl{num_labels}_data_fold{fold+1}_test.npy')
        val_label_path = os.path.join("numpy_files", f'vsl{num_labels}_label_fold{fold+1}_test.npy')

        transforms = Compose([
            Rotate(15, 80, 25, (0.5, 0.5))
        ])

        train_dataset = FeederINCLUDE(
            data_path=train_data_path,
            label_path=train_label_path,
            transform=transforms
        )
        val_dataset = FeederINCLUDE(
            data_path=val_data_path,
            label_path=val_label_path
        )

        train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
        val_dataloader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)

        model = Model(num_class=num_labels, num_point=46, num_person=1, in_channels=2,
                      graph_args={"layout": "mediapipe_two_hand", "strategy": "spatial"},
                      learning_rate=config['learning_rate'], weight_decay=config['weight_decay'])

        # Path to your checkpoint
        checkpoint_path = "/media/ibmelab/ibme21/Test/checkpoints/epoch=55-valid_loss=0.41-valid_accuracy=0.85-autsl-aagcn.ckpt"

        # Load the checkpoint
        with pl_legacy_patch():
            checkpoint = torch.load(checkpoint_path, map_location=device)

        # Get the state dict
        state_dict = checkpoint['state_dict']

        # Remove the keys for the final layer (adjust 'fc' to match your model's final layer name)
        filtered_state_dict = {k: v for k, v in state_dict.items() if not k.startswith('fc.')}

        # Load the filtered state dict into the model
        model.load_state_dict(filtered_state_dict, strict=False)

        callbacks = [
            ModelCheckpoint(
                dirpath="checkpoints",
                monitor="valid_accuracy",
                mode="max",
                every_n_epochs=2,
                #filename=f'{{epoch}}-{{valid_accuracy:.2f}}-vsl{num_labels}-aagcn-fold={fold}'
                filename=f'autsl_vsl{num_labels}-aagcn-fold={fold+1}'
            ),
        ]

        trainer = pl.Trainer(max_epochs=100, accelerator="auto", check_val_every_n_epoch=1,
                             devices=1, callbacks=callbacks)

        trainer.fit(model, train_dataloader, val_dataloader)

        # Get the best validation accuracy for this fold
        val_accuracy = trainer.callback_metrics['valid_accuracy'].item()  # Access the validation accuracy
        print(f"Fold {fold + 1} finished with validation accuracy: {val_accuracy:.4f}")

        # Update the best accuracy and fold index if this fold's accuracy is higher
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_fold = fold + 1  # Store fold number (1-indexed)

    # Print the highest accuracy and the corresponding fold
    print(f"The highest validation accuracy achieved is {best_accuracy:.4f} from fold {best_fold}.")
'''

In [ ]:
print(f"The highest validation accuracy achieved of autsl vsl{num_labels} is {best_accuracy:.4f} from fold {best_fold}.")

train based on AUTSL, based on 1 fold only to see the training process graph

In [ ]:
num_labels = 199

In [ ]:
'''
import os
import numpy as np
import torch
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from feeder import FeederINCLUDE
from aagcn import Model
from augumentation import Rotate, Compose
from pytorch_lightning.utilities.migration import pl_legacy_patch
import matplotlib.pyplot as plt

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

if __name__ == '__main__':
    k_folds = 10
    config = {'batch_size': 128, 'learning_rate': 0.0137296, 'weight_decay': 0.000150403}

    device = "cuda" if torch.cuda.is_available() else "cpu"

    best_accuracy = 0.0
    best_fold = -1
    best_fold_history = None  # To store accuracy and loss history for the best fold

    all_fold_history = []  # Store history of all folds

    for fold in range(k_folds):
        print(f"Starting fold {fold + 1}/{k_folds}")
        train_data_path = os.path.join("numpy_files", f'vsl{num_labels}_data_fold{fold+1}_train.npy')
        train_label_path = os.path.join("numpy_files", f'vsl{num_labels}_label_fold{fold+1}_train.npy')
        val_data_path = os.path.join("numpy_files", f'vsl{num_labels}_data_fold{fold+1}_test.npy')
        val_label_path = os.path.join("numpy_files", f'vsl{num_labels}_label_fold{fold+1}_test.npy')

        transforms = Compose([
            Rotate(15, 80, 25, (0.5, 0.5))
        ])

        train_dataset = FeederINCLUDE(
            data_path=train_data_path,
            label_path=train_label_path,
            transform=transforms
        )
        val_dataset = FeederINCLUDE(
            data_path=val_data_path,
            label_path=val_label_path
        )

        train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
        val_dataloader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)

        model = Model(num_class=num_labels, num_point=46, num_person=1, in_channels=2,
                      graph_args={"layout": "mediapipe_two_hand", "strategy": "spatial"},
                      learning_rate=config['learning_rate'], weight_decay=config['weight_decay'])

        # Path to your checkpoint
        checkpoint_path = "vsl199/checkpoints/epoch=55-valid_loss=0.41-valid_accuracy=0.85-autsl-aagcn.ckpt"

        # Load the checkpoint
        with pl_legacy_patch():
            checkpoint = torch.load(checkpoint_path, map_location=device)

        # Get the state dict
        state_dict = checkpoint['state_dict']

        # Remove the keys for the final layer (adjust 'fc' to match your model's final layer name)
        filtered_state_dict = {k: v for k, v in state_dict.items() if not k.startswith('fc.')}

        # Load the filtered state dict into the model
        model.load_state_dict(filtered_state_dict, strict=False)

        callbacks = [
            ModelCheckpoint(
                dirpath="checkpoints",
                monitor="valid_accuracy",
                mode="max",
                every_n_epochs=2,
                filename=f'autsl_vsl{num_labels}-aagcn-fold={fold+1}'
            ),
        ]

        trainer = pl.Trainer(max_epochs=100, accelerator="auto", check_val_every_n_epoch=1,
                             devices=1, callbacks=callbacks)

        # Track accuracy and loss history
        history = {"accuracy": [], "loss": []}
        
        trainer.fit(model, train_dataloader, val_dataloader)

        # Store accuracy and loss per epoch for this fold
        for epoch in range(trainer.max_epochs):
            # You can get the values from the trainer callback metrics
            epoch_accuracy = trainer.callback_metrics.get('valid_accuracy').item()
            epoch_loss = trainer.callback_metrics.get('valid_loss').item()
            history["accuracy"].append(epoch_accuracy)
            history["loss"].append(epoch_loss)

        all_fold_history.append(history)

        # Get the best validation accuracy for this fold
        val_accuracy = trainer.callback_metrics['valid_accuracy'].item()  # Access the validation accuracy
        print(f"Fold {fold + 1} finished with validation accuracy: {val_accuracy:.4f}")

        # Update the best accuracy and fold index if this fold's accuracy is higher
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_fold = fold + 1  # Store fold number (1-indexed)
            best_fold_history = history  # Store the history for the best fold

    # Print the highest accuracy and the corresponding fold
    print(f"The highest validation accuracy achieved is {best_accuracy:.4f} from fold {best_fold}.")

    # Now let's plot the accuracy and loss for the best fold
    if best_fold_history is not None:
        plt.figure(figsize=(12, 6))

        # Plot accuracy
        plt.subplot(1, 2, 1)
        plt.plot(best_fold_history['accuracy'], label='Validation Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.title(f'Accuracy for Fold {best_fold}')
        plt.legend()

        # Plot loss
        plt.subplot(1, 2, 2)
        plt.plot(best_fold_history['loss'], label='Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title(f'Loss for Fold {best_fold}')
        plt.legend()

        # Show plots
        plt.tight_layout()
        plt.show()
'''

In [ ]:
# Print the highest accuracy and the corresponding fold
print(f"The highest validation accuracy achieved is {best_accuracy:.4f} from fold {best_fold}.")

In [ ]:
plt.figure(figsize=(12, 6))

        # Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(best_fold_history['accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title(f'Accuracy for Fold {best_fold}')
plt.legend()

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(best_fold_history['loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title(f'Loss for Fold {best_fold}')
plt.legend()

# Show plots
plt.tight_layout()
plt.show()